In [ ]:
!nvidia-smi

Thu Jun 13 08:43:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#Chuẩn bị

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/Tieu_luan/data/'

Mounted at /content/gdrive
/content/gdrive/MyDrive/Tieu_luan/data


In [2]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
from pyvi import ViTokenizer

Mounted at /content/gdrive
/content/gdrive/MyDrive/Tieu_luan/data


#Gộp dataset

In [ ]:
df_kaggle = pd.read_csv("vietnamese_ecommerce_review.csv")
df_tiki = pd.read_csv("cmt_tiki_final.csv")

# https://www.kaggle.com/datasets/hienbm/vietnamese-ecommerce-review

In [ ]:
df_kaggle = df_kaggle[['content', 'score']]
df_tiki = df_tiki[['content', 'rating']]

df_kaggle = df_kaggle.rename(columns={'score': 'target'})
df_tiki = df_tiki.rename(columns={'rating': 'target'})

In [ ]:
print(f"Kaggle {df_kaggle.shape}")
print(f"Tiki {df_tiki.shape}")

Kaggle (1300086, 2)
Tiki (8566625, 2)


In [ ]:
df_final = pd.concat([df_kaggle, df_tiki], ignore_index=True)

In [ ]:
print(f"sau khi gộp:  {df_final.shape}")

sau khi gộp:  (9866711, 2)


In [ ]:
df_kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300086 entries, 0 to 1300085
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   content  1300018 non-null  object
 1   target   1300086 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 19.8+ MB


In [ ]:
df_tiki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8566625 entries, 0 to 8566624
Data columns (total 2 columns):
 #   Column   Dtype  
---  ------   -----  
 0   content  object 
 1   target   float64
dtypes: float64(1), object(1)
memory usage: 130.7+ MB


In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9866711 entries, 0 to 9866710
Data columns (total 2 columns):
 #   Column   Dtype  
---  ------   -----  
 0   content  object 
 1   target   float64
dtypes: float64(1), object(1)
memory usage: 150.6+ MB


In [ ]:
df_final.describe()

,target
count,9.866710e+06
mean,4.713047e+00
std,8.628372e-01
min,0.000000e+00
25%,5.000000e+00
50%,5.000000e+00
75%,5.000000e+00
max,5.000000e+00


In [ ]:
df_final.head(1000)

,content,target
0,Bán gì cũng mất,1.0
1,TÍNH CÀI APP NHƯNG ĐỌC BÌNH LUẬN THẤY NGÁN NG...,1.0
2,Ok,5.0
3,Vô trang 𝑽𝒂𝒚𝑻𝒐𝒕𝑵𝒉𝒂𝒕 để vay 0% & kiếm tiền 0 vốn,5.0
4,Vô trang KTIEN. VN để nhận free 500k,1.0
...,...,...
995,Giao hàng không đúng mẫu hàng khác mẫu trong đ...,1.0
996,Sao vô k dc z,1.0
997,Mua ở Tiki chỉ tin tưởng mỗi Tiki tranding và ...,3.0
998,Tiki càng ngày càng xuống cấp. Tài khoản của t...,1.0


#Tiền xử lý dữ liệu

In [ ]:
#Bước 1
df_final = df_final.dropna().drop_duplicates()
df_final = df_final.sample(frac=1).reset_index(drop=True)
df_final = df_final[0:700000]
df_final.shape

(695210, 2)

In [ ]:
en2vi_path = pd.read_table('/content/gdrive/MyDrive/Tieu_luan/Pre-processing/top_500_adjective_eng.txt', delimiter='\t')
en2vi_dict = dict(zip(en2vi_path['Adj'], en2vi_path['Mean']))

In [ ]:
correct_char_path = pd.read_table('/content/gdrive/MyDrive/Tieu_luan/Pre-processing/correct_char.txt', delimiter='\t')
correct_char_dict = dict(zip(correct_char_path['char'], correct_char_path['correct_char']))

en2vi_path = pd.read_table('/content/gdrive/MyDrive/Tieu_luan/Pre-processing/top_500_adjective_eng.txt', delimiter='\t')
en2vi_dict = dict(zip(en2vi_path['Adj'], en2vi_path['Mean']))

teencode_path = pd.read_table('/content/gdrive/MyDrive/Tieu_luan/Pre-processing/teencode.txt', delimiter='\t')
teencode_dict = dict(zip(teencode_path['incorrect'], teencode_path['correct']))

def preprocess_text(text):
    # Bước 2
    text = text.lower()

    # Bước 3
    def remove_special_words(text):
        if not isinstance(text, str): return ""
        text = re.sub(r'\#\w+', '', text)   # Loại bỏ các từ có hashtag (#)
        text = re.sub(r'<.*?>', '', text)   # Loại bỏ các thẻ HTML (< >)
        text = re.sub(r'http\S+', '', text)   # Loại bỏ các URL (http://)
        text = re.sub(r'@\w+', '', text)    # Loại bỏ các mention (@)
        return text

    text = remove_special_words(text)

   # Bước 4
    def remove_special_characters(text):
        text_pre = re.compile("[^a-zA-Zàáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệđíìỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵđ\s]")
        return re.sub(text_pre, ' ', text)

    def correct_char(text):
        words = text.split()
        for word in words:
            for k, v in correct_char_dict.items():
                if word == k:
                    key = ' '+k+' '
                    value = ' '+v+' '
                    text = text.replace(key, value)
        return text

    text = remove_special_characters(text)
    text = correct_char(text)


    # Bước 5
    def remove_repeated_characters(text):
        return re.sub(r'(\w)\1+', r'\1', text)

    def remove_repeated_words(text):
        words = text.split()
        seen = set()
        new_words = []
        for word in words:
            if word not in seen:
                seen.add(word)
                new_words.append(word)
        return " ".join(new_words)

    text = remove_repeated_characters(text)
    text = remove_repeated_words(text)

    # Bước 6
    def remove_emoji(text):
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"
                                   u"\U0001F300-\U0001F5FF"
                                   u"\U0001F680-\U0001F6FF"
                                   u"\U0001F1E0-\U0001F1FF"
                                   u"\U00002500-\U00002BEF"
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r' ', text)

    text = remove_emoji(text)

    # Bước 7
    def change_en_to_vn(text):

        words = text.split()
        for word in words:
            for k, v in en2vi_dict.items():
                if word == k:
                    key = ' '+k+' '
                    value = ' '+v+' '
                    text = text.replace(key, value)
        return text

    text = change_en_to_vn(text)

    # Bước 8
    def teencode(text):

        words = text.split()
        for word in words:
            for k, v in teencode_dict.items():
                if word == k:
                    key = ' '+k+' '
                    value = ' '+v+' '
                    text = text.replace(key, value)
        return text

    text = teencode(text)

    # Bước 9
    def remove_extra_spaces(text):
        return ' '.join(text.split())

    text = remove_extra_spaces(text)

    def remove_text_over_len(text):
        text = ' '.join([w for w in text.split() if len(w) <= 7])
        return text

    text = remove_text_over_len(text)

    # Bước 10
    # Tokenize tiếng Việt
    text = ViTokenizer.tokenize(text)

    return text

In [ ]:
# df_final['preprocess'] = df_final['content'].str.lower()

In [ ]:
df_final['preprocess'] = df_final['content'].map(preprocess_text)
df_final

,content,target,preprocess
0,Lazada như cc họ,1.0,lazada như c họ
1,1-Yêu cầu chạy quảng cáo giảm % tiền hàng. Chạ...,1.0,yêu_cầu chạy quảng_cáo giảm tiền hàng rồi chỉ ...
2,Hơi delay nhưng tốt,5.0,hơi hoãn nhưng tốt
3,Nhân viên giao hàng chậm đóng gói ko xứng đáng...,1.0,nhân_viên giao hàng chậm đóng_gói không xứng_đ...
4,Ko shop bán hàng như hình rất uy tín nên hôm n...,5.0,ko shop bán hàng như hình rất uy_tín nên hôm_n...
...,...,...,...
695205,"Mua hàng khá ổn giao hơi lâu tầm 4,5 là có như...",4.0,mua hàng khá ổn giao hơi lâu tầm là có như bù ...
695206,hợp voi tui tien,5.0,hợp voi tôi tien
695207,Mới biết đến Lazada nên mình cho 4sao,4.0,mới biết đến lazada nên mình cho sao
695208,"Mua món hàng trên quan được giảm 50k ,,thâ...",1.0,mua mo n ha người trên quan đươ c gia m không ...


In [ ]:
# Bước 11
sentiment = {1: 0,
            2: 0,
            3: 0,
            4: 1,
            5: 1}

df_final["target"] = df_final["target"].map(sentiment)
df_final = df_final[['preprocess','target']].dropna()
df_final = df_final.query('preprocess != ""')

In [ ]:
df_final

,preprocess,target
0,lazada như c họ,0
1,yêu_cầu chạy quảng_cáo giảm tiền hàng rồi chỉ ...,0
2,hơi hoãn nhưng tốt,1
3,nhân_viên giao hàng chậm đóng_gói không xứng_đ...,0
4,ko shop bán hàng như hình rất uy_tín nên hôm_n...,1
...,...,...
689375,mua hàng khá ổn giao hơi lâu tầm là có như bù ...,1
689376,hợp voi tôi tien,1
689377,mới biết đến lazada nên mình cho sao,1
689378,mua mo n ha người trên quan đươ c gia m không ...,0


In [ ]:
df_final['target'] = df_final['target'].astype(int)

In [ ]:
df_final.shape

(689379, 2)

In [ ]:
num = df_final['target'].value_counts()
num

target
1    394219
0    295160
Name: count, dtype: int64

In [ ]:
# Bước 12
df_final.to_csv("data_preprocessed.csv")

#Kaggle

In [ ]:
df_kaggle.shape

(1300086, 12)

In [ ]:
df_kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300086 entries, 0 to 1300085
Data columns (total 12 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   Unnamed: 0            1300086 non-null  int64 
 1   reviewid              1300086 non-null  object
 2   username              1300084 non-null  object
 3   userimage             1300086 non-null  object
 4   content               1300018 non-null  object
 5   score                 1300086 non-null  int64 
 6   thumbsupcount         1300086 non-null  int64 
 7   reviewcreatedversion  961240 non-null   object
 8   at                    1300086 non-null  object
 9   replycontent          838878 non-null   object
 10  repliedat             838878 non-null   object
 11  appid                 1300086 non-null  object
dtypes: int64(3), object(9)
memory usage: 119.0+ MB


In [ ]:
df_kaggle.describe()

,Unnamed: 0,score,thumbsupcount
count,1.300086e+06,1.300086e+06,1.300086e+06
mean,9.827248e+05,3.982997e+00,1.610502e+00
std,5.717422e+05,1.584356e+00,9.502741e+01
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.563362e+05,3.000000e+00,0.000000e+00
50%,8.813575e+05,5.000000e+00,0.000000e+00
75%,1.542192e+06,5.000000e+00,0.000000e+00
max,2.082519e+06,5.000000e+00,6.657300e+04


In [ ]:
df_kaggle = df_kaggle[['content', 'score']]
df_kaggle.head(1000)

,content,score
0,Bán gì cũng mất,1
1,TÍNH CÀI APP NHƯNG ĐỌC BÌNH LUẬN THẤY NGÁN NG...,1
2,Ok,5
3,Vô trang 𝑽𝒂𝒚𝑻𝒐𝒕𝑵𝒉𝒂𝒕 để vay 0% & kiếm tiền 0 vốn,5
4,Vô trang KTIEN. VN để nhận free 500k,1
...,...,...
995,Giao hàng không đúng mẫu hàng khác mẫu trong đ...,1
996,Sao vô k dc z,1
997,Mua ở Tiki chỉ tin tưởng mỗi Tiki tranding và ...,3
998,Tiki càng ngày càng xuống cấp. Tài khoản của t...,1


In [ ]:
df_kaggle = df_kaggle.dropna().drop_duplicates()
df_kaggle = df_kaggle.sample(frac=1).reset_index(drop=True)
df_kaggle = df_kaggle[0:700000]
df_kaggle.shape

(668963, 2)

In [ ]:
df_kaggle['preprocess'] = df_kaggle['content'].str.lower()

In [ ]:
df_kaggle['preprocess'] = df_kaggle['preprocess'].map(preprocess_text)

In [ ]:
df_kaggle

,content,score,preprocess
0,Không cho kiểm tra hàng trước khi nhận,2,không cho kiê m tra ha người trươ c khi nhâ n
1,Thú vị,2,thú_vị
2,giao hàng càng nhanh càng tốt,5,giao hàng càng nhanh tốt
3,Thích Shopee lắm có điều mỗi lần mình muốn hàn...,5,thích shopee lắm có_điều mỗi lần mình muốn hàn...
4,Mua hàng ko được luôn,1,mua hàng không được luôn
...,...,...,...
668958,Yêu thích nhanh de,3,yêu thích nhanh de
668959,Đề nghị cho ktra hàng trước khi nhận,1,đề_nghị cho kiểm_tra hàng trước khi nhận
668960,"rất hài lòng với ứng dụng này, chạy nhanh, k b...",5,rất hài_lòng với ứng_dụng này chạy nhanh không...
668961,Quảng cáo gây ức chế.lazada bị máát,1,quảng_cáo gây ức_chế lazada bị mát


In [ ]:
df_kaggle = df_kaggle.dropna().drop_duplicates()

In [ ]:
sentiment = {1: 0,
            2: 0,
            3: 0,
            4: 1,
            5: 1}

df_kaggle["target"] = df_kaggle["score"].map(sentiment)
df_kaggle = df_kaggle[['preprocess','target']].dropna()
df_kaggle = df_kaggle.query('preprocess != ""')

df_kaggle['num_word'] = [len(sentence.split()) for sentence in df_kaggle['preprocess']]

KeyError: 'score'

In [ ]:
print(df_kaggle.shape)

num = df_kaggle['target'].value_counts()
num

(663230, 3)


target
1    370818
0    292412
Name: count, dtype: int64

In [ ]:
df_kaggle['target'] = df_kaggle['target'].astype(int)
df_kaggle

,preprocess,target
0,không cho kiê m tra ha người trươ c khi nhâ n,0
1,thú_vị,0
2,giao hàng càng nhanh tốt,1
3,thích shopee lắm có_điều mỗi lần mình muốn hàn...,1
4,mua hàng không được luôn,0
...,...,...
663225,yêu thích nhanh de,0
663226,đề_nghị cho kiểm_tra hàng trước khi nhận,0
663227,rất hài_lòng với ứng_dụng này chạy nhanh không...,1
663228,quảng_cáo gây ức_chế lazada bị mát,0


In [ ]:
df_kaggle.shape

(663230, 2)

In [ ]:
df_kaggle.to_csv("data_kaggle_processed.csv")

#Tiki

In [ ]:
df_tiki.shape

(8566625, 5)

In [ ]:
df_tiki.shape

(8566625, 5)

In [ ]:
df_tiki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8566625 entries, 0 to 8566624
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   id_pro      int64  
 1   id_comment  int64  
 2   title       object 
 3   content     object 
 4   rating      float64
dtypes: float64(1), int64(2), object(2)
memory usage: 326.8+ MB


In [ ]:
df_tiki.describe()

,id_pro,id_comment,rating
count,8.566625e+06,8.566625e+06,8.566624e+06
mean,1.292403e+08,1.410276e+07,4.823841e+00
std,7.095784e+07,4.872014e+06,6.191630e-01
min,9.792030e+05,1.341334e+06,1.000000e+00
25%,7.378718e+07,1.177767e+07,5.000000e+00
50%,1.744442e+08,1.504569e+07,5.000000e+00
75%,1.840364e+08,1.820033e+07,5.000000e+00
max,2.749591e+08,1.988963e+07,5.000000e+00


In [ ]:
df_tiki = df_tiki[['content', 'rating']]
df_tiki.head(1000)

,content,rating
0,"Giao hang sieu nhanh , dt rat dep , rat ung ý ...",5.0
1,"Máy nhìn đẹp, mượt mà. Cần sử dụng lâu mới có ...",5.0
2,"Màn hình lớn, pin khá ổn, hệ điều hành ngon, t...",5.0
3,"Hàng nhận đúng như mô tả , sử dụng mới đánh gi...",5.0
4,"Điện thoại thì ok sử dụng ngon trong tầm giá, ...",5.0
...,...,...
995,"Máy giao nhanh, đúng mẫu mã, bọc kín đáo, cẩn ...",5.0
996,Máy bị lỗi kỹ thuật nhưng không thể liên hệ đư...,5.0
997,"Đã mua, unbox và kiểm tra, mọi thứ khá ok nha ...",5.0
998,Giao hàng nhanh. Hàng chuẩn chính hãng,5.0


In [ ]:
df_tiki = df_tiki.dropna().drop_duplicates()
df_tiki = df_tiki.sample(frac=1).reset_index(drop=True)
df_tiki = df_tiki[0:30000]
df_tiki.shape

(27619, 2)

In [ ]:
df_tiki['preprocess'] = df_tiki['content'].str.lower()

In [ ]:
df_tiki['preprocess'] = df_tiki['preprocess'].map(preprocess_text)

In [ ]:
df_tiki

,content,rating,preprocess
0,"Hình in sắc nét, nd thì khỏi phải bàn, tớ nghĩ...",5.0,hình in sắc nét nd thì khỏi phải bàn tớ nghĩ n...
1,"Giao hàng nhanh lắm luôn. Sách mới, không bị ả...",5.0,giao hàng nhanh lắm luôn sách mới không bị ảnh...
2,Sách đẹp lắm nhưng với một đứa không biết gì k...,5.0,sách đẹp lắm nhưng với một đứa không biết gì k...
3,Sản phẩm mới nhưng k được bao bọc nên trong qu...,3.0,sản_phẩm mới nhưng không được bao_bọc nên tron...
4,"Giao hành nhanh, sách đẹp",5.0,giao hành nhanh sách đẹp
...,...,...,...
27614,Tiki đóng cái thùng siêu khủng luôn. Giao hàng...,5.0,tiki đóng cái thùng siêu khủng luôn giao hàng ...
27615,"Giao nhanh, sách không bị lỗi",5.0,giao nhanh sách không bị lỗi
27616,"Giao nhanh, hàng chính hãng",5.0,giao nhanh hàng chính hãng
27617,Hơn 1 tháng mới giao hàng :),3.0,hơn tháng mới giao hàng


In [ ]:
df_tiki = df_tiki.dropna().drop_duplicates()

In [ ]:
sentiment = {1: 0,
            2: 0,
            3: 0,
            4: 1,
            5: 1}

df_tiki["target"] = df_tiki["rating"].map(sentiment)
df_tiki = df_tiki[['preprocess','target']].dropna()
df_tiki = df_tiki.query('preprocess != ""')

# df_tiki['num_word'] = [len(sentence.split()) for sentence in df_tiki['preprocess']]


In [ ]:
print(df_tiki.shape)

num = df_tiki['target'].value_counts()
num

(27436, 2)


target
1    24683
0     2753
Name: count, dtype: int64

In [ ]:
df_tiki['target'] = df_tiki['target'].astype(int)
df_tiki

,preprocess,target
0,hình in sắc nét nd thì khỏi phải bàn tớ nghĩ n...,1
1,giao hàng nhanh lắm luôn sách mới không bị ảnh...,1
2,sách đẹp lắm nhưng với một đứa không biết gì k...,1
3,sản_phẩm mới nhưng không được bao_bọc nên tron...,0
4,giao hành nhanh sách đẹp,1
...,...,...
27431,tiki đóng cái thùng siêu khủng luôn giao hàng ...,1
27432,giao nhanh sách không bị lỗi,1
27433,giao nhanh hàng chính hãng,1
27434,hơn tháng mới giao hàng,0


In [ ]:
df_tiki.shape

(27436, 2)

In [ ]:
df_tiki.to_csv("data_tiki_processed.csv")

#Kết hợp 2 dataset




In [ ]:
df = pd.read_csv("dataset_final.csv")


In [ ]:
# Gộp theo hàng
df_final = pd.concat([df_kaggle, df_tiki], ignore_index=True)

print(df_final)

                                               preprocess  target
0           không cho kiê m tra ha người trươ c khi nhâ n       0
1                                                  thú_vị       0
2                                giao hàng càng nhanh tốt       1
3       thích shopee lắm có_điều mỗi lần mình muốn hàn...       1
4                                mua hàng không được luôn       0
...                                                   ...     ...
690661  tiki đóng cái thùng siêu khủng luôn giao hàng ...       1
690662                       giao nhanh sách không bị lỗi       1
690663                         giao nhanh hàng chính hãng       1
690664                            hơn tháng mới giao hàng       0
690665  sách không được bọc bị móp gì cả tuy_nhiên hơi...       1

[690666 rows x 2 columns]


In [ ]:
df_final.to_csv("dataset_final.csv")

#Test

In [ ]:
# num = df_final['target'].value_counts()
# num

target
5.0    329642
1.0    211865
4.0     69026
3.0     49000
2.0     35675
0.0         2
Name: count, dtype: int64